In [1]:
import sklearn.model_selection as ms
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import numpy as np
from joblib import dump, load
from tqdm import tqdm

In [2]:
# Input

# This is the output table from the part of the data-post-production repo that merges
# preprocessed results. 
# Source: data-post-production/01-merge-results/01_merge_preprocessed_results
path_inference_data = "g2022_adid_01062021_11082022_text.csv"

# This is the output table from merging "g2022_adid_01062021_11082022_text.csv" and
#  "g2022_adid_01062021_11082022_var1.csv"
# Source: data-post-production/01-merge-results/01_merge_preprocessed_results
path_inference_data_vars = "google2022_adidlevel.csv"

path_model = "models/party_clf_facebook_and_google_2020.joblib"
path_model_smooth = "models/party_clf_facebook_and_google_2020_smooth.joblib"

# Output
path_predictions = "data/google/party_predictions_google_2022_both_model.csv.gz"

In [3]:
# Inference dataset
df = pd.read_csv(path_inference_data, encoding='UTF-8', keep_default_na = False, dtype = 'str')
print(df.columns)

Index(['ad_id', 'wmp_creative_id', 'ad_type', 'ad_title', 'google_asr_text',
       'aws_ocr_video_text', 'aws_ocr_img_text', 'csum_agg', 'advertiser_id',
       'advertiser_name', 'ad_text'],
      dtype='object')


In [4]:
# All fields
cols = ['advertiser_name', 'ad_title', 'google_asr_text', 'aws_ocr_video_text', 'aws_ocr_img_text', 'ad_text']
# Combine and clean up
df['combined'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df['combined'] = df['combined'].str.strip() # Remove whitesaces at end of string
df['combined'] = df['combined'].str.replace(' +', ' ', regex = True) # Remove double (and triple etc.) whitespaces inside
df = df[['ad_id', 'combined']]
# Deduplicate by text to save time during inference
df = df.groupby(['combined'])['ad_id'].apply(list)
df = df.to_frame().reset_index()
# Remove empty ads
df = df[df['combined'] != ""]

In [5]:
# Regular model
# Load model
with tqdm(total=1, desc='Loading model') as pbar:
    clf = load(path_model)
    pbar.update(1)

# Predicted probabilities
with tqdm(total=len(df), desc='Predicting probabilities') as pbar:
    pp = clf.predict_proba(df['combined'])
    df['prob_dem'] = pp[:, 0]
    df['prob_rep'] = pp[:, 1]
    pbar.update(len(df))

Predicting probabilities: 100%|████████████████████████████████████████████████| 58297/58297 [00:09<00:00, 5892.50it/s]


In [6]:
# Smooth model
# Load model
with tqdm(total=1, desc='Loading smooth model') as pbar:
    clf_smooth = load(path_model_smooth)
    pbar.update(1)

# Predicted probabilities
with tqdm(total=len(df), desc='Predicting smooth probabilities') as pbar:
    pp = clf_smooth.predict_proba(df['combined'])
    df['prob_dem_smooth'] = pp[:, 0]
    df['prob_rep_smooth'] = pp[:, 1]
    pbar.update(len(df))

Predicting smooth probabilities: 100%|█████████████████████████████████████████| 58297/58297 [00:09<00:00, 5893.12it/s]


In [7]:
df = df.explode('ad_id')

# Keep only the relevant variables
df = df[['ad_id', 'prob_dem', 'prob_rep', 'prob_dem_smooth', 'prob_rep_smooth']]

# Save
df.to_csv(path_predictions, index = False)